In [1]:
import osmnx as ox
import requests
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import warnings
import time
import math

In [4]:
amenities = gpd.read_file(r"D:\EconNet\Paris\POI_City\amenities_shops.shp")

In [12]:
amenities.groupby('amenity').count().iloc[:,1].sort_values()

amenity
cinema           91
university      120
dentist         164
college         259
doctors         270
theatre         273
greengrocer     348
butcher         437
supermarket     773
pharmacy        943
bakery         1239
cafe           2060
restaurant     7938
Name: osmid, dtype: int64

In [2]:
%%time
city = 'Paris'
# Extract bike networks according to this list
Bike_ParisC = ox.graph_from_place(['Paris','Saint-Mandé, Ile-de-France'], network_type = "bike", 
                                            retain_all = True, clean_periphery = True)

C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))
C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


Wall time: 23.8 s


In [3]:
%%time
# Get the nodes and edges from the graph
nodes, edges = ox.graph_to_gdfs(Bike_ParisC)

Wall time: 3.23 s


In [4]:
%%time
# Find cycleway if present
edges['highway'] = np.where(edges['highway'].isin(['cycleway']),'cycleway',edges['highway'])

# Unlist items if needed. First item (as most important) counts
for j in range(len(edges.columns)):
    for i in range(len(edges)):
        if isinstance(edges.iloc[i,j], list):
            edges.iloc[i,j] = edges.iloc[i,j][0]
edges

Wall time: 18 s


osmid                 name      highway  \
u           v           key                                                 
125730      268675130   0     315818975  Rue Henri Duvernois  residential   
            4761621113  0      31093720    Rue Louis Lumière  residential   
            244455548   0      85735969    Rue Louis Lumière  residential   
            1089076050  0     266463585      Rue Louis Ganne  residential   
125742      60912980    0       4039102      Rue de Buzenval  residential   
...                                 ...                  ...          ...   
10125758487 2467619666  0     938727007    Rue de l'Évangile  residential   
            10125771122 0    1106545619                  NaN      service   
10125771122 10125758487 0    1106545619                  NaN      service   
            10125771122 0    1106545620                  NaN      service   
10125771128 10125771123 0    1106545621                  NaN      service   

                            maxspeed  oneway    length lanes  \
u           v           key                                    
125730      268675130   0         30   False     8.707   NaN   
            4761621113  0         30   False   132.800     2   
            244455548   0         30   False   105.925     2   
            1089076050  0         30   False   124.290     2   
125742      60912980    0         30    True   131.074   NaN   
...                              ...     ...       ...   ...   
10125758487 2467619666  0         30    True   157.205   NaN   
            10125771122 0        NaN   False   648.253   NaN   
10125771122 10125758487 0        NaN   False   648.253   NaN   
            10125771122 0        NaN    True  1033.422   NaN   
10125771128 10125771123 0        NaN    True   147.307   NaN   

                                                                      geometry  \
u           v           key                                                      
125730      268675130   0      LINESTRING (2.41073 48.86352, 2.41083 48.86348)   
            4761621113  0    LINESTRING (2.41073 48.86352, 2.41046 48.86425...   
            244455548   0    LINESTRING (2.41073 48.86352, 2.41077 48.86337...   
            1089076050  0    LINESTRING (2.41073 48.86352, 2.41061 48.86351...   
125742      60912980    0    LINESTRING (2.40137 48.85185, 2.40134 48.85193...   
...                                                                        ...   
10125758487 2467619666  0    LINESTRING (2.36674 48.89456, 2.36793 48.89485...   
            10125771122 0    LINESTRING (2.36674 48.89456, 2.36681 48.89454...   
10125771122 10125758487 0    LINESTRING (2.36551 48.89072, 2.36553 48.89077...   
            10125771122 0    LINESTRING (2.36551 48.89072, 2.36553 48.89061...   
10125771128 10125771123 0    LINESTRING (2.36397 48.88919, 2.36436 48.88969...   

                            junction width bridge        service access  \
u           v           key                                               
125730      268675130   0        NaN   NaN    NaN            NaN    NaN   
            4761621113  0        NaN   NaN    NaN            NaN    NaN   
            244455548   0        NaN   NaN    NaN            NaN    NaN   
            1089076050  0        NaN   NaN    NaN            NaN    NaN   
125742      60912980    0        NaN   NaN    NaN            NaN    NaN   
...                              ...   ...    ...            ...    ...   
10125758487 2467619666  0        NaN   NaN    NaN            NaN    NaN   
            10125771122 0        NaN   NaN    NaN            NaN     no   
10125771122 10125758487 0        NaN   NaN    NaN            NaN     no   
            10125771122 0        NaN   NaN    NaN            NaN     no   
10125771128 10125771123 0        NaN   NaN    NaN  parking_aisle    NaN   

                            tunnel est_width  ref  
u           v           key                        
125730      268675130   0      NaN       NaN  NaN  


In [5]:
# Removing roads equal to or faster than 90 km/h
ed2 = edges[~edges['maxspeed'].isin(['walk','FR:walk','signals'])]
ed2 = pd.concat([ed2[(ed2['maxspeed'].astype(float) >= 90) == False],
                 edges[edges['maxspeed'].isin(['walk','FR:walk','signals'])]])

# Make a dedicated "number-based" speed column
speed = ed2['maxspeed']
speed = pd.Series(np.where(speed.isin(['walk','FR:walk']),10,np.where(speed.isin(['signals']),50,speed)), dtype = float)
speed.index = ed2.index

# Transform oneway and lanes to numberwise columns
ed2['cleanspeed'] = speed
ed2['oneway'] = ed2['oneway'].astype(int)
ed2['lanes'] = pd.Series(np.where(ed2['lanes'] =='Allée des petits clos',1,ed2['lanes']),index = ed2.index, dtype =float)
ed2

osmid                    name      highway  \
u           v          key                                                    
125730      268675130  0     315818975     Rue Henri Duvernois  residential   
            4761621113 0      31093720       Rue Louis Lumière  residential   
            244455548  0      85735969       Rue Louis Lumière  residential   
            1089076050 0     266463585         Rue Louis Ganne  residential   
125742      60912980   0       4039102         Rue de Buzenval  residential   
...                                ...                     ...          ...   
7112415800  2349048619 0     226044313  Place de la République   pedestrian   
8707929645  1764526274 0      41675737          Rue Montmartre  residential   
9868004812  25581574   0       4263235     Rue de la Jussienne   pedestrian   
10000480189 5900331100 0      24396069   Place Raoul Follereau   pedestrian   
            5520660517 0    1096400097   Place Raoul Follereau   pedestrian   

                           maxspeed  oneway   length  lanes  \
u           v          key                                    
125730      268675130  0         30       0    8.707    NaN   
            4761621113 0         30       0  132.800    2.0   
            244455548  0         30       0  105.925    2.0   
            1089076050 0         30       0  124.290    2.0   
125742      60912980   0         30       1  131.074    NaN   
...                             ...     ...      ...    ...   
7112415800  2349048619 0       walk       0  116.980    NaN   
8707929645  1764526274 0       walk       0    9.807    NaN   
9868004812  25581574   0       walk       0   82.039    NaN   
10000480189 5900331100 0       walk       0   44.561    NaN   
            5520660517 0       walk       0   38.586    NaN   

                                                                     geometry  \
u           v          key                                                      
125730      268675130  0      LINESTRING (2.41073 48.86352, 2.41083 48.86348)   
            4761621113 0    LINESTRING (2.41073 48.86352, 2.41046 48.86425...   
            244455548  0    LINESTRING (2.41073 48.86352, 2.41077 48.86337...   
            1089076050 0    LINESTRING (2.41073 48.86352, 2.41061 48.86351...   
125742      60912980   0    LINESTRING (2.40137 48.85185, 2.40134 48.85193...   
...                                                                       ...   
7112415800  2349048619 0    LINESTRING (2.36548 48.86702, 2.36548 48.86705...   
8707929645  1764526274 0    LINESTRING (2.34418 48.86690, 2.34421 48.86685...   
9868004812  25581574   0    LINESTRING (2.34403 48.86506, 2.34408 48.86513...   
10000480189 5900331100 0      LINESTRING (2.36351 48.87656, 2.36405 48.87637)   
            5520660517 0    LINESTRING (2.36351 48.87656, 2.36339 48.87661...   

                           junction width bridge service access tunnel  \
u           v          key                                               
125730      268675130  0        NaN   NaN    NaN     NaN    NaN    NaN   
            4761621113 0        NaN   NaN    NaN     NaN    NaN    NaN   
            244455548  0        NaN   NaN    NaN     NaN    NaN    NaN   
            1089076050 0        NaN   NaN    NaN     NaN    NaN    NaN   
125742      60912980   0        NaN   NaN    NaN     NaN    NaN    NaN   
...                             ...   ...    ...     ...    ...    ...   
7112415800  2349048619 0        NaN   NaN    NaN     NaN    NaN    NaN   
8707929645  1764526274 0        NaN   NaN    NaN     NaN    NaN    NaN   
9868004812  25581574   0        NaN   NaN    NaN     NaN    NaN    NaN   
10000480189 5900331100 0        NaN   NaN    NaN     NaN    NaN    NaN   
            5520660517 0        NaN   NaN    NaN     NaN    NaN    NaN   

                           est_width  ref  cleanspeed  
u           v          key                             
125730      268675130  0         NaN  NaN        30.0  


In [6]:
%%time
# Set nodes location to meters (for later filter in R)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
nodes['geom_x_m'] = nodes['geometry'].to_crs(3043).x
nodes['geom_y_m'] = nodes['geometry'].to_crs(3043).y

nodes['osmid_var'] = nodes.index
nodes['osmid_var'] = nodes['osmid_var'].astype('int64')

# Get edge coordinates
c1 = list()
c2 = list()
c3 = list()
c4 = list()
geom = ed2.to_crs(3043).geometry
for i in range(len(ed2)):
    l1 = list(geom.iloc[i].coords)
    c1.append(l1[0][0])
    c2.append(l1[0][1])
    c3.append(l1[-1][0])
    c4.append(l1[-1][1])

# Add edge coordinates to df
ed2['x1_m'] = c1
ed2['y1_m'] = c2
ed2['x2_m'] = c3
ed2['y2_m'] = c4

# Make if path not exists and save the files there.
warnings.filterwarnings('ignore')
if not os.path.exists('D:/EconNet/'+str(city)+'/Cycling_City/'):
    os.makedirs('D:/EconNet/'+str(city)+'/Cycling_City/')
nodes.to_file('D:/EconNet/'+str(city)+'/Cycling_City/BikePoints.shp')
ed2.to_file('D:/EconNet/'+str(city)+'/Cycling_City/BikeRoutes.shp')
nodes.sort_values('osmid')

Wall time: 20.9 s


,y,x,street_count,highway,ref,geometry,geom_x_m,geom_y_m,osmid_var
osmid,,,,,,,,,
125730,48.9,2.4,4,NaN,NaN,POINT (2.41073 48.86352),456781.9,5412451.9,125730
125742,48.9,2.4,4,NaN,NaN,POINT (2.40137 48.85185),456085.3,5411159.8,125742
125747,48.9,2.4,3,NaN,NaN,POINT (2.41466 48.85426),457062.4,5411420.0,125747
125760,48.8,2.4,3,NaN,NaN,POINT (2.40126 48.82951),456057.3,5408676.5,125760
360930,48.8,2.4,3,NaN,NaN,POINT (2.36150 48.82635),453136.4,5408348.3,360930
...,...,...,...,...,...,...,...,...,...
10124076562,48.9,2.4,3,NaN,NaN,POINT (2.38138 48.85035),454617.1,5411004.3,10124076562
10125758487,48.9,2.4,3,NaN,NaN,POINT (2.36674 48.89456),453584.4,5415927.6,10125758487
10125771122,48.9,2.4,3,NaN,NaN,POINT (2.36551 48.89072),453490.7,5415502.2,10125771122


In [47]:
# Extract transit access points and bike stations from Apur (open source data Ile-de-France)
warnings.filterwarnings('ignore')
transit_access = gpd.read_file(r"D:\EconNet\acces.geojson")
bike_stations = gpd.read_file(r"D:\EconNet\velib-emplacement-des-stations.geojson")

# Take the Paris boundaries
Paris_bounds = ox.geocode_to_gdf(['Paris','Saint-Mandé, Ile-de-France'])
Paris_bounds

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((2.22412 48.85420, 2.22412 48.85402, ...",48.902156,48.815576,2.469760,2.224122,297417241,relation,7444,48.858890,2.320041,"Paris, Ile-de-France, Metropolitan France, France",boundary,administrative,0.941710
1,"POLYGON ((2.41123 48.83387, 2.41143 48.83383, ...",48.849392,48.833571,2.428322,2.411228,297484937,relation,108318,48.843134,2.416118,"Saint-Mandé, Nogent-sur-Marne, Val-de-Marne, I...",boundary,administrative,0.987129


In [67]:
warnings.filterwarnings('ignore')

# Overlay city boundaries with the bike stations, only include ones inside the selected area
Paris_transitaccess = transit_access.overlay(Paris_bounds)

# Format to fit in df of amenities, take geoms x-y coordinates.
Paris_transitaccess['name'] = Paris_transitaccess['fournisseurname']+'-'+Paris_transitaccess['accname']
Paris_transitaccess['amenity'] = 'transit_entry'
Paris_transitaccess['osmid'] = Paris_transitaccess['accid']
Paris_transitaccess = Paris_transitaccess[['osmid','amenity','name','geometry']]
Paris_transitaccess['geometry'] = Paris_transitaccess.centroid
Paris_transitaccess['geom_x_m'] = Paris_transitaccess['geometry'].to_crs(3043).x
Paris_transitaccess['geom_y_m'] = Paris_transitaccess['geometry'].to_crs(3043).y
Paris_transitaccess.to_file('D:/EconNet/Paris/POI_City/OSM_transit_entry.shp')
Paris_transitaccess

,osmid,amenity,name,geometry,geom_x_m,geom_y_m
0,50147909,transit_entry,RATP-av. des Ternes,POINT (2.29757 48.87817),448497.378945,5.414151e+06
1,50147917,transit_entry,RATP-Entrée / Sortie,POINT (2.34065 48.86895),451647.168192,5.413097e+06
2,50147989,transit_entry,RATP-r. de Rivoli,POINT (2.33584 48.86242),451288.035029,5.412374e+06
3,50147988,transit_entry,RATP-pl. du Palais Royal,POINT (2.33626 48.86256),451319.031831,5.412390e+06
4,50147774,transit_entry,RATP-Terre-plein,POINT (2.35797 48.87598),452924.057800,5.413868e+06
...,...,...,...,...,...,...
848,50148006,transit_entry,RATP-av. Foch,POINT (2.41881 48.84626),457359.873079,5.410528e+06
849,50148000,transit_entry,RATP-Hôpital Bégin,POINT (2.42795 48.84544),458029.670540,5.410432e+06
850,50148005,transit_entry,RATP-r. du Parc,POINT (2.42000 48.84614),457446.846611,5.410515e+06
851,50148007,transit_entry,RATP-av. du Gén. de Gaulle,POINT (2.41727 48.84635),457246.905830,5.410539e+06


In [70]:
warnings.filterwarnings('ignore')

# Overlay city boundaries with the bike stations, only include ones inside the selected area
Paris_bikestations = bike_stations.overlay(Paris_bounds)

# Format to fit in df of amenities, take geoms x-y coordinates.
Paris_bikestations['amenity'] = 'bikestation'
Paris_bikestations['osmid'] = Paris_bikestations['stationcode']
Paris_bikestations = Paris_bikestations[['osmid','amenity','name','geometry']]
Paris_bikestations['geometry'] = Paris_bikestations.centroid
Paris_bikestations['geom_x_m'] = Paris_bikestations['geometry'].to_crs(3043).x
Paris_bikestations['geom_y_m'] = Paris_bikestations['geometry'].to_crs(3043).y
Paris_bikestations.to_file('D:/EconNet/Paris/POI_City/OSM_bikestations.shp')
Paris_bikestations

,osmid,amenity,name,geometry,geom_x_m,geom_y_m
0,9020,bikestation,Toudouze - Clauzel,POINT (2.33736 48.87930),451415.978458,5.414249e+06
1,7001,bikestation,Invalides - Duroc,POINT (2.31619 48.84779),449832.495243,5.410761e+06
2,9002,bikestation,Abbeville - Faubourg Poissonnière,POINT (2.34915 48.87922),452280.107639,5.414234e+06
3,19003,bikestation,Quai de la Seine,POINT (2.37039 48.88449),453842.769604,5.414807e+06
4,11023,bikestation,Saint-Maur - Roquette,POINT (2.38355 48.85869),454784.282202,5.411930e+06
...,...,...,...,...,...,...
997,41602,bikestation,Place Charles Digeon,POINT (2.41843 48.84354),457329.314123,5.410226e+06
998,12041,bikestation,Bois de Vincennes.,POINT (2.41322 48.83381),456939.069339,5.409147e+06
999,41606,bikestation,Pasteur - Gambetta,POINT (2.42360 48.84573),457710.774142,5.410466e+06
1000,41604,bikestation,Lagny - Joffre,POINT (2.41795 48.84931),457299.186291,5.410867e+06


In [7]:
%%time
# suppress scientific notation of geometries
pd.set_option('display.float_format', lambda x: '%.1f' % x)
start_time = time.time()

# Amenities and shops that need to be extracted
s = ['restaurant','cafe','college','university','pharmacy','dentist','doctors','cinema','theatre','supermarket',
            'bakery','butcher','greengrocer']
m = ['amenity','amenity','amenity','amenity','amenity','amenity','amenity','amenity','amenity','shop','shop','shop','shop']
Paris_amenities = pd.DataFrame()

if not os.path.exists('D:/EconNet/'+str(city)+'/POI_City/'):
    os.makedirs('D:/EconNet/'+str(city)+'/POI_City/')
    
# For every amenity/shop, extract from OSM
for i in range(len(s)):
    warnings.filterwarnings('ignore')
    locals()[s[i]] = ox.geometries_from_place(['Paris','Saint-Mandé, Ile-de-France'], tags={m[i]:s[i]})
    
    # Subset and rename columns
    locals()[s[i]] = locals()[s[i]][[m[i],'name','geometry']]
    locals()[s[i]].columns = ['amenity','name','geometry']
    print('path: D:/EconNet/'+str(city)+'/POI_City/OSM_'+m[i]+'_'+s[i]+'.shp')
    
    # Take the centroid of the location for simpler computation in R
    warnings.filterwarnings('ignore')
    locals()[s[i]]['geometry'] = locals()[s[i]]['geometry'].centroid
    
    # Get X and Y coordinates for computation in R
    locals()[s[i]]['geom_x_m'] = locals()[s[i]]['geometry'].to_crs(3043).x
    locals()[s[i]]['geom_y_m'] = locals()[s[i]]['geometry'].to_crs(3043).y
    
    # Save the specific amenity as shapefile
    locals()[s[i]].to_file('D:/EconNet/'+str(city)+'/POI_City/OSM_'+m[i]+'_'+s[i]+'.shp')
    
    # Add the amenity to the mastertable of amenities/shops
    Paris_amenities = pd.concat([Paris_amenities,locals()[s[i]]])
    print(s[i],'done',round((time.time() - start_time) / 60,2),' mns')

# Format and export the mastertable
Paris_amenities = pd.concat([Paris_amenities,Paris_bikestations,Paris_transitaccess])
Paris_amenities['element_ty'] = Paris_amenities['element_ty'].fillna('node')
Paris_amenities = Paris_amenities.reset_index()
Paris_amenities = Paris_amenities.iloc[:,2:]
Paris_amenities = Paris_amenities.drop_duplicates()
Paris_amenities.to_file('D:/EconNet/Paris/POI_City/amenities_shops.shp')
Paris_amenities

path: D:/EconNet/Paris/POI_City/OSM_amenity_restaurant.shp
restaurant done 1.18  mns
path: D:/EconNet/Paris/POI_City/OSM_amenity_cafe.shp
cafe done 2.31  mns
path: D:/EconNet/Paris/POI_City/OSM_amenity_college.shp
college done 3.4  mns
path: D:/EconNet/Paris/POI_City/OSM_amenity_university.shp
university done 4.56  mns
path: D:/EconNet/Paris/POI_City/OSM_amenity_pharmacy.shp
pharmacy done 9.01  mns
path: D:/EconNet/Paris/POI_City/OSM_amenity_dentist.shp
dentist done 10.15  mns
path: D:/EconNet/Paris/POI_City/OSM_amenity_doctors.shp
doctors done 11.24  mns
path: D:/EconNet/Paris/POI_City/OSM_amenity_cinema.shp
cinema done 15.58  mns
path: D:/EconNet/Paris/POI_City/OSM_amenity_theatre.shp
theatre done 16.64  mns
path: D:/EconNet/Paris/POI_City/OSM_shop_supermarket.shp
supermarket done 17.96  mns
path: D:/EconNet/Paris/POI_City/OSM_shop_bakery.shp
bakery done 19.07  mns
path: D:/EconNet/Paris/POI_City/OSM_shop_butcher.shp
butcher done 20.13  mns
path: D:/EconNet/Paris/POI_City/OSM_shop_gr

amenity                  name  \
element_type osmid                                          
node         158603712   restaurant  The Village Terrazza   
             175539450   restaurant             Le Severo   
             247439841   restaurant                   Uno   
             247455916   restaurant      La Crypte Polska   
             247455920   restaurant       Mystery Cuisine   
...                             ...                   ...   
way          685428046  greengrocer                   NaN   
             868010246  greengrocer                   NaN   
             868010250  greengrocer   Ferme Saint Quentin   
             868010254  greengrocer          Épicerie bio   
relation     14146134   greengrocer      Au bout du champ   

                                        geometry  geom_x_m  geom_y_m  
element_type osmid                                                    
node         158603712  POINT (2.32287 48.82984)  450304.4 5408760.7  
             175539450  POINT (2.32453 48.83199)  450428.9 5408999.3  
             247439841  POINT (2.34350 48.86185)  451849.4 5412306.2  
             247455916  POINT (2.32555 48.86729)  450538.3 5412922.8  
             247455920  POINT (2.33724 48.86590)  451394.3 5412760.4  
...                                          ...       ...       ...  
way          685428046  POINT (2.32617 48.87570)  450591.7 5413857.3  
             868010246  POINT (2.35514 48.87688)  452717.0 5413969.9  
             868010250  POINT (2.35521 48.87680)  452722.6 5413960.4  
             868010254  POINT (2.35493 48.87679)  452701.5 5413960.4  
relation     14146134   POINT (2.32031 48.84095)  450127.9 5409997.8  

[14915 rows x 5 columns]

In [115]:
Paris_amenities.groupby('amenity').count().iloc[:,1].sort_values()

amenity
cinema             91
university        120
dentist           164
college           259
doctors           270
theatre           273
greengrocer       348
butcher           437
supermarket       773
transit_entry     853
pharmacy          943
bikestation      1002
bakery           1239
cafe             2060
restaurant       7938
Name: osmid, dtype: int64